https://www.kaggle.com/c/competitive-data-science-final-project

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
ls all

ls: all: No such file or directory


In [4]:
path = '/Users/rizki/Dropbox/Coursera/AML_HowToKaggle/FinalProject/PredictSales/all/'

In [5]:
items = pd.read_csv(path+'items.csv')
item_categories = pd.read_csv(path+'item_categories.csv')
shops = pd.read_csv(path+'shops.csv')
sales_train = pd.read_csv(path+'sales_train.csv')
test = pd.read_csv(path+'test.csv')

In [6]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


# Counting

In [7]:
print 'total rows in train set', sales_train.shape[0]
unique_items = items['item_id'].unique().size
print 'Number of unique item_id :', unique_items
unique_shops = shops['shop_id'].unique().size
print 'Number of unique shop_id :', unique_shops
total_possible_shop_id_item_id = unique_shops*unique_items
print 'total_possible_shop_id_item_id =',total_possible_shop_id_item_id
print 'shop_item combinations in test set:', test.shape[0]
print 'shop_item combinations in train set:', sales_train.groupby(['shop_id','item_id']).count().shape[0]

total rows in train set 2935849
Number of unique item_id : 22170
Number of unique shop_id : 60
total_possible_shop_id_item_id = 1330200
shop_item combinations in test set: 214200
shop_item combinations in train set: 424124


# Format 'date' to datetime

In [ ]:
sales_train['date'] = pd.to_datetime(sales_train['date'],format='%d.%m.%Y')
sales_train[sales_train['date'].dt.month==1].head()

# Add item_category_id to sales_train

In [ ]:
items.head()

In [ ]:
sales_train.sort_values(by='item_id').head()

In [ ]:
sales_train = pd.merge(sales_train,items[['item_id','item_category_id']],on='item_id',how='left').sort_values(by='item_id')
sales_train.head()

# Check by total item_cnt by month per year

In [ ]:
print 'years in train data: ',sales_train['date'].dt.year.unique()

In [ ]:
sales_train['year'] = sales_train['date'].dt.year.astype('string')
sales_train['month'] = sales_train['date'].dt.month.astype('string')
# sales_train['Y_M'] = sales_train['year']+'_'+sales_train['month']
# sales_train.drop(columns=['year','month'], inplace=True)

In [ ]:
sales_train_year = sales_train['date'].dt.year.astype('string')
sales_train_month = sales_train['date'].dt.month.astype('string')
sales_train['Y_M'] = sales_train_year+'_'+sales_train_month

In [ ]:
sales_train.head()

In [ ]:
sales_train.groupby(['date_block_num','Y_M'])['item_cnt_day'].agg(['sum']).head()

In [ ]:
#split by year
sales_train_2013 = sales_train[sales_train['year']=='2013']
sales_train_2014 = sales_train[sales_train['year']=='2014']
sales_train_2015 = sales_train[sales_train['year']=='2015']

In [ ]:
total_item_cnt_2013 = sales_train_2013.groupby(['date_block_num','Y_M'])['item_cnt_day'].sum().values
total_item_cnt_2014 = sales_train_2014.groupby(['date_block_num','Y_M'])['item_cnt_day'].sum().values
total_item_cnt_2015 = sales_train_2015.groupby(['date_block_num','Y_M'])['item_cnt_day'].sum().values

In [ ]:
import sys
WorkDir = '/Users/rizki/Dropbox/Coursera/AML_HowToKaggle/FinalProject/PredictSales/'
sys.path.insert(0, WorkDir)
import utility.utility as ut

In [ ]:
## absolute
total_item_cnt_2013 = (total_item_cnt_2013)
total_item_cnt_2014 = (total_item_cnt_2014)
total_item_cnt_2015 = (total_item_cnt_2015)

y_test = test
y_test['item_cnt_month'] = 0.32
saveName='constant_0p32'
#saveplots=True

yearlySales = [total_item_cnt_2013,total_item_cnt_2014,total_item_cnt_2015]

ut.plotYearly(yearlySales,y_test,saveName,False)
ut.plotYearly_v2(yearlySales,y_test,saveName,False)
ut.plotResidual(yearlySales,y_test,saveName,False)

In [ ]:
#translation by mean
total_item_cnt_2013_trans = (total_item_cnt_2013-total_item_cnt_2013.mean())
total_item_cnt_2014_trans = (total_item_cnt_2014-total_item_cnt_2014.mean())
total_item_cnt_2015_trans = (total_item_cnt_2015-total_item_cnt_2015.mean())

y_test = test
y_test['item_cnt_month'] = 0.32
saveName='constant_0p32'
#saveplots=True

yearlySales = [total_item_cnt_2013_trans,total_item_cnt_2014_trans,total_item_cnt_2015_trans]

ut.plotYearly(yearlySales,y_test,saveName,False,total_item_cnt_2015.mean())
ut.plotYearly_v2(yearlySales,y_test,saveName,False,total_item_cnt_2015.mean())
ut.plotResidual(yearlySales,y_test,saveName,False,total_item_cnt_2015.mean())

In [ ]:
#plot item_cnt vs month, per shop_id
shop_id_2013 = sales_train_2013.groupby(['shop_id'],as_index=False)['item_cnt_day'].sum()['shop_id'].values
shop_id_2014 = sales_train_2014.groupby(['shop_id'],as_index=False)['item_cnt_day'].sum()['shop_id'].values
shop_id_2015 = sales_train_2015.groupby(['shop_id'],as_index=False)['item_cnt_day'].sum()['shop_id'].values

# Missing shops 

In [ ]:
missing_shop_id_2013 = set(shops['shop_id']) - set(shop_id_2013)
missing_shop_id_2014 = set(shops['shop_id']) - set(shop_id_2014)
missing_shop_id_2015 = set(shops['shop_id']) - set(shop_id_2015)

In [ ]:
print 'missing_shop_id_2013:',list(missing_shop_id_2013)
print 'missing_shop_id_2014:',list(missing_shop_id_2014)
print 'missing_shop_id_2015:',list(missing_shop_id_2015)

In [ ]:
print 'shops in 2013 which closed in 2014:',list(set(shop_id_2013)-set(shop_id_2014))
print 'shops opened in 2014 not present in 2013:',list(set(shop_id_2014)-set(shop_id_2013))
print 'shops in 2014 which closed in 2015:',list(set(shop_id_2014)-set(shop_id_2015))
print 'shops opened in 2015 not present in 2014:',list(set(shop_id_2015)-set(shop_id_2014))

In [ ]:
shop_in_train_not_in_test = set(sales_train['shop_id'].unique())-set(test['shop_id'].unique())
print 'shop_in_train_not_in_test:',list(shop_in_train_not_in_test)

In [ ]:
shop_in_test_not_in_train = set(test['shop_id'].unique())-set(sales_train['shop_id'].unique())
print 'shop_in_test_not_in_train:',list(shop_in_test_not_in_train)

# Missing items

In [ ]:
missing_item_id_2013 = set(items['item_id'].unique()) - set(sales_train[sales_train.date.dt.year==2013]['item_id'].unique())
missing_item_id_2014 = set(items['item_id'].unique()) - set(sales_train[sales_train.date.dt.year==2014]['item_id'].unique())
missing_item_id_2015 = set(items['item_id'].unique()) - set(sales_train[sales_train.date.dt.year==2015]['item_id'].unique())

In [ ]:
print 'missing_item_id_2013:',len(missing_item_id_2013), 'items'
print 'missing_item_id_2014:',len(missing_item_id_2014), 'items'
print 'missing_item_id_2015:',len(missing_item_id_2015),'item'

In [ ]:
item_in_train_not_in_test = set(sales_train['item_id'].unique())-set(test['item_id'].unique())
print 'item_in_train_not_in_test:',len(item_in_train_not_in_test), 'items'

In [ ]:
item_in_test_not_in_train = set(test['item_id'].unique()) -set(sales_train['item_id'].unique()) 
print 'item_in_test_not_in_train:',len(item_in_test_not_in_train), 'items'

# Missing  shop_id_item_id combination

In [ ]:
sales_train_sample = sales_train.groupby(['shop_id','item_id'],as_index=False).count()[['shop_id','item_id']]
sales_train_sample['shop_item_id'] = sales_train_sample['shop_id'].astype('string')+'_'+sales_train_sample['item_id'].astype('string')
sales_train_shop_item_id = sales_train_sample['shop_item_id'].values
sales_train_sample.head()

In [ ]:
test_sample = test[['shop_id','item_id']].sort_values(by=['shop_id','item_id'])
test_sample['shop_item_id'] = test_sample['shop_id'].astype('string') +'_'+ test_sample['item_id'].astype('string')
test_shop_item_id = test_sample['shop_item_id'].values
test_sample.head()

In [ ]:
shop_item_id_in_test_not_in_train = set(test_shop_item_id)-set(sales_train_shop_item_id)
print 'shop_item_id_in_test_not_in_train : ',len(shop_item_id_in_test_not_in_train)